In [ ]:
import ROOT
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import sys
import multiprocessing as mp
import uproot
import pandas as pd
import pickle
import h5py
import gzip
import math
import timeit


#infile = "../NTuples/production_sample_v2.root"
infile = "../NTuples/production_ntuples_v2.root"

In [ ]:

inFileROOT = ROOT.TFile.Open(infile, "READ")
#h_tot_pot = inFileROOT.Get("TotalPOT")
h_tot_pot = inFileROOT.Get("TOTPOT_Clone")
TOT_POT = h_tot_pot.GetBinContent(1)
inFileROOT.Close()
TOT_POT = f"{TOT_POT:.2e}"
print("Total POT", TOT_POT)


inFile = uproot.open(infile)

#h_tot_pot = ROOT.TFile.Open(infile).Get("TotalPOT")
#pot = h_tot_pot.GetBinContent(1)

slc_tree = inFile["slc_truth_tree"]
particle_tree1 = inFile["particle_tree1"]
particle_tree2 = inFile["particle_tree2"]
daughter_tree1 = inFile["daughter_tree1"]
daughter_tree2 = inFile["daughter_tree2"]
cosmic_tree1 = inFile["cosmic_tree1"]
cosmic_tree2 = inFile["cosmic_tree2"]

slc_df = slc_tree.arrays(slc_tree.keys(), library="pd")
particle_df1 = particle_tree1.arrays(particle_tree1.keys(), library="pd")
particle_df2 = particle_tree2.arrays(particle_tree2.keys(), library="pd")
daughter_df1 = daughter_tree1.arrays(daughter_tree1.keys(), library="pd")
daughter_df2 = daughter_tree2.arrays(daughter_tree2.keys(), library="pd")
cosmic_df1 = cosmic_tree1.arrays(cosmic_tree1.keys(), library="pd")
cosmic_df2 = cosmic_tree2.arrays(cosmic_tree2.keys(), library="pd")

slc_df[:2]

In [ ]:
particle_df1[:2]

In [ ]:
particle_df2[:2]

In [ ]:
daughter_df1[:2]

In [ ]:
daughter_df2[:2]

In [ ]:
cosmic_df1[:2]

In [ ]:
cosmic_df2[:2]

In [ ]:
cosmic_df1["KEEP"] = 0
condition = cosmic_df1.index.isin(cosmic_df1.query("pdg == 13 or pdg == -13").index)
cosmic_df1.loc[condition, "KEEP"] = 1
cosmic_df2["KEEP"] = cosmic_df1["KEEP"]

cosmic_df1 = cosmic_df1.query("KEEP == 1")
cosmic_df2 = cosmic_df2.query("KEEP == 1")

cosmic_df1["start_x"] = cosmic_df2["start_x"]
cosmic_df1["start_y"] = cosmic_df2["start_y"]
cosmic_df1["start_z"] = cosmic_df2["start_z"]
cosmic_df1["end_x"] = cosmic_df2["end_x"]
cosmic_df1["end_y"] = cosmic_df2["end_y"]
cosmic_df1["end_z"] = cosmic_df2["end_z"]

cosmic_df1[:2]

In [ ]:
def is_inTPC(row):
    x, y, z = row["vtx_x"], row["vtx_y"], row["vtx_z"]
    if math.isnan(x) or math.isnan(y) or math.isnan(z):
        return 0
    #if (x == 'NaN') or (y == 'NaN') or (z == 'NaN'):
    #    return 0
        
    if (-200 < x < 200) and (-200 < y < 200) and (0 < z < 500):
        return 1
    else:
        return 0

slc_df["inTPC"] = slc_df.apply(is_inTPC, axis=1)
slc_df[:2]

In [ ]:
topology_labels = {
    0:r"$\nu_{\mu} CC$",
    1:r"$\nu_{\mu} NC$",
    2:r"$\nu_{e} CC$",
    3:r"$\nu_{e} NC$",
    4:r"$\bar{\nu}_{\mu}$",
    5:r"$\bar{\nu}_{e}$",
    6:r"DIRT $\nu$",
    7:"Cosmic",
}

topology_selections = {
    0:"pdg == 14.0 and iscc == 1.0 and inTPC == 1",
    1:"pdg == 14.0 and isnc == 1.0 and inTPC == 1",
    2:"pdg == 12.0 and iscc == 1.0 and inTPC == 1",
    3:"pdg == 12.0 and isnc == 1.0 and inTPC == 1",
    4:"pdg == -14.0 and inTPC == 1",
    5:"pdg == -12.0 and inTPC == 1",
    6:"(pdg == 14.0 or pdg == 12.0 or pdg == -14.0 or pdg == -12.0 ) and inTPC == 0",
    7:"pdg == -1",
}

In [ ]:
# Let's look at the file composition

N = slc_df.shape[0]

bin_centers = [1]
bin_width = 1
N_prev = 0
#plt.yscale('log')
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    
    plt.bar(bin_centers, [N_top], width=bin_width, bottom=N_prev, alpha=0.6, label=topology_labels[num])
    N_prev += N_top

plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.xticks([])
plt.ylabel("Slice Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
# Let's add the topologies to the dataframes

# initialize to -1 for now
slc_df["TOP"] = -1

for num in range(len(topology_selections.keys())):
    #temp = slc_df.query(topology_selections[num])
    condition = slc_df.index.isin(slc_df.query(topology_selections[num]).index)
    slc_df.loc[condition, "TOP"] = num
    
slc_df[:2]


In [ ]:


#df_result = df_large.merge(df_small, on=['col1', 'col2'], how='left')

# Let's try to add the topology to the particle dfs
#df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP', 'vtx_x', 'vtx_y', 'vtx_z']]
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'TOP']]
particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
#particle_df2 = particle_df2.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

particle_df1[:2]

In [ ]:
daughter_df1 = daughter_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

daughter_df1[:2]

In [ ]:
# Try grabbing the true neutrino vertex for the particle df
df_small_filtered = slc_df[['run', 'subrun', 'evt', 'slc', 'vtx_x', 'vtx_y', 'vtx_z']]
particle_df1 = particle_df1.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')
#particle_df2 = particle_df2.merge(df_small_filtered, on=['run', 'subrun', 'evt', 'slc'], how='left')

particle_df1[:2]

In [ ]:
particle_df2["TOP"] = particle_df1["TOP"]
daughter_df2["TOP"] = daughter_df1["TOP"]

particle_df2[:2]

In [ ]:
# Load the CRT histograms and make some geometry
WEST_X = 383
EAST_X = -383
SOUTH_Z = -179.05 
NORTH_Z = 779.85
FLAT_Y = -378.8
TOPLOW_Y = 632
TOPHIGH_Y = 767


In [ ]:
# implement the TPC Geometry 

NORTH_TPC_Z = 500
SOUTH_TPC_Z = 0
EAST_TPC_X = -200
WEST_TPC_X = 200
TOP_TPC_Y = 200
BOTT_TPC_Y = -200


def cross_top_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOP_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOP_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_bottom_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (BOTT_TPC_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (BOTT_TPC_Y - bzy)/mzy
    if (-200 < x_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0


def cross_south_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0
        
def cross_north_tpc(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_TPC_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_TPC_Z + byz
    if (-200 < x_int < 200) and (-200 < y_int < 200):
        return 1
    else:
        return 0


def cross_east_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0

def cross_west_tpc(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_TPC_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_TPC_X - bxz)/mxz
    if (-200 < y_int < 200) and (0 < z_int < 500):
        return 1
    else:
        return 0



def cross_tpc(start, dir):
    top = cross_top_tpc(start, dir)
    bott = cross_bottom_tpc(start, dir)
    north = cross_north_tpc(start, dir)
    south = cross_south_tpc(start, dir)
    east = cross_east_tpc(start, dir)
    west = cross_west_tpc(start, dir)
    if top or bott or north or south or east or west:
        return 1
    else:
        return 0
        

In [ ]:
# We need to check if a muon that has an endpoint in the TPC has an endpoint that terminates on the TPC Boundary
# This will determine if we should consider propogating it further to see if it hit more CRT Walls

def is_muon_crossing_tpc(row):
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]
    start_x, start_y, start_z = row["start_x"], row["start_y"], row["start_z"]

    if (-202 <= end_x <= 202) and (-202 <= end_y <= 202) and (-2 <= end_z <= 502):
        if (-202 <= end_y <= -198):
            # terminates on the bottom tpc somewhere
            return 1
        if (-2 <= end_z <= 2) or (498 <= end_z <= 502):
            # terminates on the north or south tpc somewhere
            return 1
        if (-202 <= end_x <= -198) or (198 <= end_x <= 202):
            # terminates on the east or west tpc
            return 1
            
        # Doesn't terminate on the tpc boundary of interest    
        return 0
    else:
        # Doesn't touch the tpc at all
        return 1
        

In [ ]:
def check_tpc_crossing(row):
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]
    if (-200 < end_x < 200) and (-200 < end_y < 200) and (0 < end_z < 500):
        return 1
        
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
    dir = [row["pc"]/norm, row["py"]/norm, row["pz"]/norm]
    cross = cross_tpc(start, dir)
    return cross
    
cosmic_df1["cross_tpc"] = cosmic_df1.apply(check_tpc_crossing, axis=1)
cosmic_df1[:2]

In [ ]:

bin_centers = [0, 1]
bin_width = 1
N_prev_sel = 0
N_prev_rej = 0

N_cross = cosmic_df1.query("cross_tpc == 1").shape[0]
N_not_cross = cosmic_df1.query("cross_tpc == 0").shape[0]
    
plt.bar(bin_centers, [N_cross, N_not_cross], width=bin_width, alpha=0.6, label="Cosmic Muons")

#plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.legend()
#plt.xticks([])
plt.xticks([0, 1], ['Crosses TPC', 'No TPC'])
plt.ylabel("Muon Count", fontsize=14)
#plt.ylim([0, N])
#plt.title("All Primary Particles")
plt.show()

In [ ]:
# Load the CRT histograms

crt_hist_file = ROOT.TFile.Open("/Users/alexanderantonakis/Desktop/crt_heatmaps.root", "READ")

h_tophigh = crt_hist_file.Get("h_tophigh_strip")
h_toplow = crt_hist_file.Get("h_toplow_strip")
h_north = crt_hist_file.Get("h_north_strip")
h_south = crt_hist_file.Get("h_south_strip")
h_east = crt_hist_file.Get("h_east_strip")
h_west = crt_hist_file.Get("h_west_strip")
h_flat = crt_hist_file.Get("h_flat_strip")

c = ROOT.TCanvas("c", "c", 700, 500)
h_flat.Draw("Colz")
c.Draw()

In [ ]:
# Implement the CRT geometry

def cross_crt_tophigh(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPHIGH_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPHIGH_Y - bzy)/mzy
    binx = h_tophigh.GetXaxis().FindBin(z_int)
    biny = h_tophigh.GetYaxis().FindBin(x_int)
    if h_tophigh.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_toplow(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (TOPLOW_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (TOPLOW_Y - bzy)/mzy
    binx = h_toplow.GetXaxis().FindBin(z_int)
    biny = h_toplow.GetYaxis().FindBin(x_int)
    if h_toplow.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_south(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*SOUTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*SOUTH_Z + byz
    binx = h_south.GetXaxis().FindBin(x_int)
    biny = h_south.GetYaxis().FindBin(y_int)
    if h_south.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_north(start, dir):
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    x_int = mxz*NORTH_Z + bxz
    myz = dir[1]/dir[2]
    byz = start[1] - myz*start[2]
    y_int = myz*NORTH_Z + byz
    binx = h_north.GetXaxis().FindBin(x_int)
    biny = h_north.GetYaxis().FindBin(y_int)
    if h_north.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0

def cross_crt_east(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*EAST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (EAST_X - bxz)/mxz
    binx = h_east.GetXaxis().FindBin(z_int)
    biny = h_east.GetYaxis().FindBin(y_int)
    if h_east.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_west(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    y_int = mxy*WEST_X + bxy
    mxz = dir[0]/dir[2]
    bxz = start[0] - mxz*start[2]
    z_int = (WEST_X - bxz)/mxz
    binx = h_west.GetXaxis().FindBin(z_int)
    biny = h_west.GetYaxis().FindBin(y_int)
    if h_west.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt_flat(start, dir):
    mxy = dir[1]/dir[0]
    bxy = start[1] - mxy*start[0]
    x_int = (FLAT_Y - bxy)/mxy
    mzy = dir[1]/dir[2]
    bzy = start[1] - mzy*start[2]
    z_int = (FLAT_Y - bzy)/mzy
    binx = h_flat.GetXaxis().FindBin(z_int)
    biny = h_flat.GetYaxis().FindBin(x_int)
    
    if h_flat.GetBinContent(binx, biny) > 0:
        return 1
    else:
        return 0


def cross_crt(start, dir):
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    num_c = c_tophigh + c_toplow + c_north + c_south + c_east + c_west + c_flat
    return num_c
    
print("made crt functions")

In [ ]:
cosmic_df1["crossing_muon"] = cosmic_df1.apply(is_muon_crossing_tpc, axis=1)
cosmic_df1[:2]

In [ ]:
def check_crt_crossing(row):

    # check if the muon terminates on the tpc boundary
    muon_crossing = is_muon_crossing_tpc(row)
    
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]      
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
    dir = [row["pc"]/norm, row["py"]/norm, row["pz"]/norm]
    cross = 0
    if muon_crossing == 1:
        cross = cross_crt(start, dir)
        return cross
    else:
        # Need to see how many CRT walls were crossed before the end point
        delx = end_x - start[0]
        dely = end_y - start[1]
        delz = end_z - start[2]
        norm = (delx**2 + dely**2 + delz**2)**0.5
        dir = [delx/norm, dely/norm, delz/norm]
        c_tophigh = cross_crt_tophigh(start, dir)
        c_toplow = cross_crt_toplow(start, dir)
        c_north = cross_crt_north(start, dir)
        c_south = cross_crt_south(start, dir)
        c_east = cross_crt_east(start, dir)
        c_west = cross_crt_west(start, dir)
        c = 0
        if start[0] > WEST_X and end_x < WEST_X:
            # could hit west
            c += c_west
            
        if start[0] < EAST_X and end_x > EAST_X:
            c += c_east 
            # could hit east
        
        if start[2] < SOUTH_Z and end_z > SOUTH_Z:
            c += c_south

        if start[2] > NORTH_Z and end_z < NORTH_Z:
            c+= c_north
        c += c_tophigh
        c += c_toplow
        return c

cosmic_df1["cross_crt"] = cosmic_df1.apply(check_crt_crossing, axis=1)
cosmic_df1[:2]

In [ ]:
N_cosmic_muon_inTPC = cosmic_df1.query("cross_tpc == 1").shape[0]

N_cosmic_muon_inCRT = cosmic_df1.query("cross_crt > 0 and cross_tpc == 1").shape[0]


plt.errorbar([1], [N_cosmic_muon_inTPC], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inTPC), fmt="o", c="black", label="Crosses TPC")
plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.legend(loc="center")
plt.xticks([])
plt.ylabel("Cosmic Muon Count/"+str(TOT_POT) + " POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
def cross_crt_eff(row):
    c = int(row["cross_crt"])
    count = 0
    for num in range(c): 
        r = np.random.uniform()
        if r <= 0.95:
            count += 1
    return count
    
    
cosmic_df1["cross_crt_eff"] = cosmic_df1.apply(cross_crt_eff, axis=1)   
cosmic_df1[:2]

In [ ]:
N_cosmic_muon_inCRT_eff = cosmic_df1.query("cross_crt_eff > 0 and cross_tpc == 1").shape[0]
N_cosmic_muon_inCRT_eff_track = cosmic_df1.query("cross_crt_eff > 1 and cross_tpc == 1").shape[0]

plt.errorbar([1], [N_cosmic_muon_inTPC], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inTPC), fmt="o", c="black", label="Crosses TPC")
plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT_eff), fmt="o", c="red", label="Crosses CRT: 95%")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff_track], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT_eff), fmt="o", c="green", label="Crosses CRT: 95% (N > 1)")
plt.legend(loc="center")
plt.xticks([])
plt.ylabel("Cosmic Muon Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.title("Cosmics Interacting with TPC", fontsize=20)
plt.ylim([180000, 480000])
plt.show()



plt.errorbar([1], [1.0*N_cosmic_muon_inCRT/N_cosmic_muon_inTPC], xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label="Crosses CRT")
#plt.errorbar([1], [N_cosmic_muon_inCRT], xerr=[0.5], yerr=np.sqrt(N_cosmic_muon_inCRT), fmt="o", c="blue", label="Crosses CRT")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff/N_cosmic_muon_inTPC], xerr=[0.5], yerr=[0.0], fmt="o", c="red", label="Crosses CRT: 95%")
plt.errorbar([1], [N_cosmic_muon_inCRT_eff_track/N_cosmic_muon_inTPC], xerr=[0.5], yerr=[0.0], fmt="o", c="green", label="Crosses CRT: 95% (N > 1)")
plt.legend(loc="lower center")
plt.xticks([])
plt.ylim([0, 1])
plt.title("Cosmics Interacting with TPC", fontsize=20)
plt.ylabel("Cosmic Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
import matplotlib.patches as patches

def plot_cosmic(row):
    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]  
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
    dir = [row["pc"]/norm, row["py"]/norm, row["pz"]/norm]
    cross_tpc = row["cross_tpc"]
    cross_crt = row["cross_crt"]
    cross_muon = row["crossing_muon"]
    
    #if math.isnan(end_x) or math.isnan(end_y) or math.isnan(end_z):
    if end_x < -8000.0 or end_y < -8000.0 or end_z < -8000.0:
        end_y = -400
        byz = start[1] - (dir[1]/dir[2])*start[2]
        bxy = start[1] - (dir[1]/dir[0])*start[0]
        end_z = (end_y - byz) / (dir[1]/dir[2])
        end_x = (end_y - bxy) / (dir[1]/dir[0])

    # Create a figure with two subplots side by side
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    # Create a rectangle (x, y, width, height)
    rectxy = patches.Rectangle((-200, -200), 400, 400, linewidth=1, edgecolor='red', facecolor='red', alpha=0.5)
    rectzy = patches.Rectangle((0, -200), 500, 400, linewidth=1, edgecolor='red', facecolor='red', alpha=0.5)

    
    # Add the rectangle to the plot
    axes[0].add_patch(rectzy)
    axes[1].add_patch(rectxy)

    # Add CRT to YZ Projection
    axes[0].plot([SOUTH_Z, SOUTH_Z], [h_south.GetYaxis().GetXmin(), h_south.GetYaxis().GetXmax()], color="g")
    axes[0].plot([NORTH_Z, NORTH_Z], [h_north.GetYaxis().GetXmin(), h_north.GetYaxis().GetXmax()], color="g")
    axes[0].plot([h_toplow.GetXaxis().GetXmin(), h_toplow.GetXaxis().GetXmax()], [TOPLOW_Y, TOPLOW_Y], color="g")
    axes[0].plot([h_tophigh.GetXaxis().GetXmin(), h_tophigh.GetXaxis().GetXmax()], [TOPHIGH_Y, TOPHIGH_Y], color="g")
    axes[0].plot([h_flat.GetXaxis().GetXmin(), h_flat.GetXaxis().GetXmax()], [FLAT_Y, FLAT_Y], color="g")

    eastyz = patches.Rectangle((h_east.GetXaxis().GetXmin(), h_east.GetYaxis().GetXmin()), 
                               h_east.GetXaxis().GetXmax() - h_east.GetXaxis().GetXmin(), 
                               h_east.GetYaxis().GetXmax() - h_east.GetYaxis().GetXmin(), 
                               linewidth=1, edgecolor='green', facecolor='green', alpha=0.2)
    axes[0].add_patch(eastyz)

    # Add CRT to XY Projection
    axes[1].plot([EAST_X, EAST_X], [h_east.GetYaxis().GetXmin(), h_east.GetYaxis().GetXmax()], color="g")
    axes[1].plot([WEST_X, WEST_X], [h_west.GetYaxis().GetXmin(), h_west.GetYaxis().GetXmax()], color="g")
    axes[1].plot([h_flat.GetYaxis().GetXmin(), h_flat.GetYaxis().GetXmax()], [FLAT_Y, FLAT_Y], color="g")
    axes[1].plot([h_toplow.GetYaxis().GetXmin(), h_toplow.GetYaxis().GetXmax()], [TOPLOW_Y, TOPLOW_Y], color="g")
    axes[1].plot([h_tophigh.GetYaxis().GetXmin(), h_tophigh.GetYaxis().GetXmax()], [TOPHIGH_Y, TOPHIGH_Y], color="g")

    southxy = patches.Rectangle((h_south.GetXaxis().GetXmin(), h_south.GetYaxis().GetXmin()), 
                               h_south.GetXaxis().GetXmax() - h_south.GetXaxis().GetXmin(), 
                               h_south.GetYaxis().GetXmax() - h_south.GetYaxis().GetXmin(), 
                               linewidth=1, edgecolor='green', facecolor='green', alpha=0.2)
    axes[1].add_patch(southxy)
    
    c = "blue"
    if cross_crt < 2:
        c="red"
    # Plot on the first subplot YZ projection
    axes[0].plot([start[2], end_z], [start[1], end_y], color=c)
    axes[0].set_title("Cross TPC: "+str(cross_tpc) + " Cross CRT: "+str(cross_crt) + " CMuon "+ str(cross_muon))
    axes[0].set_xlabel("True Z [cm]")
    axes[0].set_ylabel("True Y [cm]")
    #axes[0].legend()

    # Plot on the second subplot XY projection
    axes[1].plot([start[0], end_x], [start[1], end_y], color=c)
    axes[1].set_title("Cross TPC: "+str(cross_tpc) + " Cross CRT: "+str(cross_crt) + " CMuon "+ str(cross_muon))
    axes[1].set_xlabel("True X [cm]")
    axes[1].set_ylabel("True Y [cm]")

    # Adjust layout and show the plot
    plt.tight_layout()  # Prevent overlap between subplots
    plt.show()

print("Made Cosmic Plotter")

In [ ]:

for num in range(20):
    plot_cosmic(cosmic_df1.iloc[num])



# Investigate Muon Neutrino CC interactions

In [ ]:
particle_df2["pdg"] = particle_df1["pdg"]

muons_numucc1 = particle_df1.query("TOP == 0 and pdg == 13")
muons_numucc2 = particle_df2.query("TOP == 0 and pdg == 13")

muons_numucc2[:2]

In [ ]:
plt.hist(muons_numucc2["end_x"].values, bins=100, histtype="step", linewidth=2, label="X")
plt.hist(muons_numucc2["end_y"].values, bins=100, histtype="step", linewidth=2, label="Y")
plt.hist(muons_numucc2["end_z"].values, bins=100, histtype="step", linewidth=2, label="Z")
plt.xlabel("True Muon End Point [cm]")
#plt.xlim([-11000, -8000])
#plt.yscale("log")
plt.title(r"$\nu_{\mu}$ CC Primary Muons")
plt.legend()
plt.show()

In [ ]:
plt.hist(muons_numucc2.query("end_x < -8000")["end_y"].values, bins=100, histtype="step", linewidth=2, label="Y")
plt.hist(muons_numucc2.query("end_x < -8000")["end_z"].values, bins=100, histtype="step", linewidth=2, label="Z")
plt.title(r"$\nu_{\mu}$ CC Primary Muons (True End X < -8000 cm)")
plt.xlabel("True Muon End Point [cm]")
#plt.ylim([0, 2])
#plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
def inTPC_point(point):
    if (-200 <= point[0] <= 200) and (-200 <= point[1] <= 200) and (0 <= point[2] <= 500):
        return 1
    else:
        return 0

def isTPC_Contained(row):
    s = [row["start_x"], row["start_y"], row["start_z"]]
    e = [row["end_x"], row["end_y"], row["end_z"]]
    if inTPC_point(s) and inTPC_point(e):
        return 1
    else:
        return 0
        
        
muons_numucc2["TPC_Contained"] = muons_numucc2.apply(isTPC_Contained, axis=1)
muons_numucc2[:2]


In [ ]:
plt.hist(muons_numucc2.query("TPC_Contained == 0")["end_x"].values, bins=100, histtype="step", linewidth=2, label="X")
plt.hist(muons_numucc2.query("TPC_Contained == 0")["end_y"].values, bins=100, histtype="step", linewidth=2, label="Y")
plt.hist(muons_numucc2.query("TPC_Contained == 0")["end_z"].values, bins=100, histtype="step", linewidth=2, label="Z")
plt.xlabel("True Muon End Point (Not Contained in TPC)[cm]")
#plt.xlim([-11000, -8000])
#plt.yscale("log")
plt.title(r"$\nu_{\mu}$ CC Primary Muons")
plt.legend()
plt.show()

In [ ]:
plt.errorbar([1], [1.0*muons_numucc2.query("TPC_Contained == 1").shape[0]/muons_numucc2.shape[0]], 
             xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label=r"$\nu_{\mu}$ CC")

plt.legend(loc="lower center", fontsize=20)
plt.xticks([])
#plt.ylim([0, 1])
plt.title("Muons Contained in TPC/Total", fontsize=20)
plt.ylabel("Primary Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
muons_numucc2["pc"] = muons_numucc1["px"]
muons_numucc2["py"] = muons_numucc1["py"]
muons_numucc2["pz"] = muons_numucc1["pz"]

def check_crt_crossing_nu(row):

    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]      
    start = [row["start_x"], row["start_y"], row["start_z"]]
    norm = 1.
    dir = 1.
    if end_x < -8000 and end_y < -8000 and end_z < -8000:
        norm = np.sqrt(row["pc"]**2 + row["py"]**2 + row["pz"]**2)
        if norm == 0.0:
            norm = 1.0
        dir = np.array([row["pc"]/norm, row["py"]/norm, row["pz"]/norm])
    else:
        # Need to see how many CRT walls were crossed before the end point
        delx = end_x - start[0]
        dely = end_y - start[1]
        delz = end_z - start[2]
        norm = (delx**2 + dely**2 + delz**2)**0.5
        if norm == 0.0:
            norm = 1.0
        dir = np.array([delx/norm, dely/norm, delz/norm])
        
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    c_wall = c_north + c_south + c_west + c_east + c_flat
    c_up = c_tophigh + c_toplow
    if c_wall > 0:
        return 1
    else:
        return c_up


muons_numucc2["cross_crt"] = muons_numucc2.apply(check_crt_crossing_nu, axis=1)
muons_numucc2[:2]

In [ ]:
muons_numucc2["cross_crt_eff"] = muons_numucc2.apply(cross_crt_eff, axis=1)
N_numucc_muons_out = muons_numucc2.query("TPC_Contained == 0").shape[0]
N_numucc_muons_out_crt = muons_numucc2.query("TPC_Contained == 0 and cross_crt > 0").shape[0]
N_numucc_muons_out_crt_eff = muons_numucc2.query("TPC_Contained == 0 and cross_crt_eff > 0").shape[0]
N_numucc_muons_out_crt_eff_track = muons_numucc2.query("TPC_Contained == 0 and cross_crt_eff > 1").shape[0]

plt.errorbar([1], [N_numucc_muons_out], xerr=[0.5], yerr=[N_numucc_muons_out**0.5], fmt="o", c="black", label=r"$\nu_{\mu}$ CC Not Contained")
plt.errorbar([1], [N_numucc_muons_out_crt], xerr=[0.5], yerr=[N_numucc_muons_out_crt**0.5], fmt="o", c="blue", label="Passes through CRT")
plt.errorbar([1], [N_numucc_muons_out_crt_eff], xerr=[0.5], yerr=[N_numucc_muons_out_crt_eff**0.5], fmt="o", c="red", label="95%")
plt.errorbar([1], [N_numucc_muons_out_crt_eff_track], xerr=[0.5], yerr=[N_numucc_muons_out_crt_eff_track**0.5], fmt="o", c="green", label="95% (N > 1)")

plt.legend(loc="center", fontsize=14)
plt.xticks([])
#plt.ylim([0, 1])
plt.title(r"$\nu_{\mu}$ CC Muons Not Contained in TPC", fontsize=20)
plt.ylabel("Primary Muon Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
#plt.errorbar([1], [N_numucc_muons_out], xerr=[0.5], yerr=[N_numucc_muons_out**0.5], fmt="o", c="black", label=r"$\nu_{\mu}$ CC Not Contained")
plt.errorbar([1], [N_numucc_muons_out_crt/N_numucc_muons_out], xerr=[0.5], yerr=[0.0], 
             fmt="o", c="blue", label="Passes through CRT")
plt.errorbar([1], [N_numucc_muons_out_crt_eff/N_numucc_muons_out], xerr=[0.5], yerr=[0.0], 
             fmt="o", c="red", label="95%")
plt.errorbar([1], [N_numucc_muons_out_crt_eff_track/N_numucc_muons_out], xerr=[0.5], yerr=[0.0], 
             fmt="o", c="green", label="95% (N > 1)")

plt.legend(loc="upper center", fontsize=14)
plt.xticks([])
plt.ylim([0, 1])
plt.title(r"$\nu_{\mu}$ CC Muons Not Contained in TPC", fontsize=20)
plt.ylabel("Primary Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
muons_numucc2["crossing_muon"] = muons_numucc2.apply(is_muon_crossing_tpc, axis=1)
muons_numucc2[:2]

In [ ]:
muons_numucc2["cross_tpc"] = 1
muons_numucc2[:2]

In [ ]:
for num in range(20):
    plot_cosmic(muons_numucc2.query("TPC_Contained == 0").iloc[num])

# Investigate Dirt Muons Entering the TPC

In [ ]:
# Need to add the dirt nu vertex to get the true start point
particle_df1[:2]

In [ ]:
muons_dirt1 = particle_df1.query("TOP == 6 and pdg == 13")
muons_dirt2 = particle_df2.query("TOP == 6 and pdg == 13")

muons_dirt2[:2]

In [ ]:

def endpoint_inTPC(row):
    ex, ey, ez = row["end_x"], row["end_y"], row["end_z"]
    if (-200 <= ex <= 200) and (-200 <= ey <= 200) and (0 <= ez <= 500):
        return 1
    else:
        return 0

muons_dirt2["inTPC"] = muons_dirt2.apply(endpoint_inTPC, axis=1)

muons_dirt2_enters = muons_dirt2.query("inTPC == 1")
muons_dirt2_enters[:2]

In [ ]:
plt.hist(muons_dirt2_enters["start_x"].values, bins=100, histtype="step", linewidth=2, label="X")
plt.hist(muons_dirt2_enters["start_y"].values, bins=100, histtype="step", linewidth=2, label="Y")
plt.hist(muons_dirt2_enters["start_z"].values, bins=100, histtype="step", linewidth=2, label="Z")
plt.xlabel("True Muon Start Point [cm]", fontsize=14)
plt.title("Dirt Muons Entering TPC")
plt.legend()
plt.show()

In [ ]:
N_dirt_enters = muons_dirt2_enters.shape[0]
N_dirt = muons_dirt2.shape[0]

plt.errorbar([1], [N_dirt], xerr=[0.5], yerr=[N_dirt**0.5], fmt="o", c="black", label="Dirt Muons")
plt.errorbar([1], [N_dirt_enters], xerr=[0.5], yerr=[N_dirt_enters**0.5], fmt="o", c="blue", label="Endpoint in TPC")

plt.legend(loc="center", fontsize=14)
plt.xticks([])
#plt.ylim([0, 1])
plt.title("Dirt Muons", fontsize=20)
plt.ylabel("Muon Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

plt.errorbar([1], [N_dirt_enters/N_dirt], xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label="Dirt Endpoint in TPC/Tot")

plt.legend(loc="center", fontsize=14)
plt.xticks([])
plt.ylim([0, 1])
plt.title("Dirt Muons", fontsize=20)
plt.ylabel("Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:
muons_dirt2["px"] = muons_dirt1["px"]
muons_dirt2["py"] = muons_dirt1["py"]
muons_dirt2["pz"] = muons_dirt1["pz"]

muons_dirt2["vtx_x"] = muons_dirt1["vtx_x"]
muons_dirt2["vtx_y"] = muons_dirt1["vtx_y"]
muons_dirt2["vtx_z"] = muons_dirt1["vtx_z"]

def check_crt_crossing_dirt(row):

    end_x, end_y, end_z = row["end_x"], row["end_y"], row["end_z"]      
    start = [row["vtx_x"], row["vtx_y"], row["vtx_z"]]

    end_inTPC = row["inTPC"]
    
    norm = 1.
    dir = 1.
    if not end_inTPC:
        norm = np.sqrt(row["px"]**2 + row["py"]**2 + row["pz"]**2)
        if norm == 0.0:
            norm = 1.0
        dir = np.array([row["px"]/norm, row["py"]/norm, row["pz"]/norm])
    else:
        # Need to see how many CRT walls were crossed before the end point
        delx = end_x - start[0]
        dely = end_y - start[1]
        delz = end_z - start[2]
        norm = (delx**2 + dely**2 + delz**2)**0.5
        if norm == 0.0:
            norm = 1.0
        dir = np.array([delx/norm, dely/norm, delz/norm])
    
    c_tophigh = cross_crt_tophigh(start, dir)
    c_toplow = cross_crt_toplow(start, dir)
    c_north = cross_crt_north(start, dir)
    c_south = cross_crt_south(start, dir)
    c_east = cross_crt_east(start, dir)
    c_west = cross_crt_west(start, dir)
    c_flat = cross_crt_flat(start, dir)
    cmax = c_tophigh + c_toplow + c_east + c_west + c_north + c_south + c_flat
    
    c = 0
    if end_inTPC:
        # We definitely go through the TPC
        if start[1] > TOPHIGH_Y:
            c += c_tophigh
            c+= c_toplow
            
        if start[1] > TOPLOW_Y:
            c += c_toplow
            
        if start[2] < SOUTH_Z:
            c += c_south
            if start[0] < EAST_X:
                c += c_west
                
            if start[0] > WEST_X:
                c+= c_west

        elif start[2] > NORTH_Z:
            c+= c_north
            if start[0] < EAST_X:
                c += c_west
                
            if start[0] > WEST_X:
                c+= c_west
        if SOUTH_Z < start[2] < NORTH_Z:
            if start[0] < EAST_X:
                c += c_west
                
            if start[0] > WEST_X:
                c+= c_west
        if start[0] < FLAT_Y:
            return (c_east + c_west + c_north + c_south + c_flat)
        return c

    # Endpoint Not Contained in TPC
    else:
        if start[1] > TOPHIGH_Y and dir[1] < 0:
            return cmax

        if start[1] > TOPLOW_Y and dir[1] < 0:
            return (c_toplow + c_east + c_west + c_north + c_south + c_flat)

        if start[1] < FLAT_Y:
            return cmax
            
        if start[1] > FLAT_Y:
            return (c_tophigh + c_toplow + c_east + c_west + c_north + c_south)
        


muons_dirt2["cross_crt"] = muons_dirt2.apply(check_crt_crossing_dirt, axis=1)
muons_dirt2[:2]

In [ ]:
N_dirt_enters_crt = muons_dirt2.query("inTPC == 1 and cross_crt > 0").shape[0]
N_dirt_enters_crt_track = muons_dirt2.query("inTPC == 1 and cross_crt > 1").shape[0]

plt.errorbar([1], [N_dirt_enters], xerr=[0.5], yerr=[N_dirt_enters**0.5], fmt="o", c="black", label="Dirt Muon Endpoint in TPC")
plt.errorbar([1], [N_dirt_enters_crt], xerr=[0.5], yerr=[N_dirt_enters_crt**0.5], fmt="o", c="blue", label="CRT")
plt.errorbar([1], [N_dirt_enters_crt_track], xerr=[0.5], yerr=[N_dirt_enters_crt_track**0.5], fmt="o", c="green", label="CRT (N > 1)")

plt.legend(loc="center", fontsize=14)
plt.xticks([])
#plt.ylim([0, 1])
plt.title("Dirt Muons", fontsize=20)
plt.ylabel("Muon Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()


plt.errorbar([1], [N_dirt_enters_crt/N_dirt_enters], xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label="CRT")
plt.errorbar([1], [N_dirt_enters_crt_track/N_dirt_enters], xerr=[0.5], yerr=[0.0], fmt="o", c="green", label="CRT (N > 1)")
plt.legend(loc="center", fontsize=14)
plt.xticks([])
plt.ylim([0, 1])
plt.title("Dirt Muons", fontsize=20)
plt.ylabel("Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()


In [ ]:
N_dirt_out = muons_dirt2.query("inTPC == 0").shape[0]
N_dirt_out_crt = muons_dirt2.query("inTPC == 0 and cross_crt > 0").shape[0]
N_dirt_out_crt_track = muons_dirt2.query("inTPC == 0 and cross_crt > 1").shape[0]

plt.errorbar([1], [N_dirt_out], xerr=[0.5], yerr=[N_dirt_out**0.5], fmt="o", c="black", label="Dirt Muon Endpoint out of TPC")
plt.errorbar([1], [N_dirt_out_crt], xerr=[0.5], yerr=[N_dirt_out_crt**0.5], fmt="o", c="blue", label="CRT")
plt.errorbar([1], [N_dirt_out_crt_track], xerr=[0.5], yerr=[N_dirt_out_crt_track**0.5], fmt="o", c="green", label="CRT (N > 1)")

plt.legend(loc="center", fontsize=14)
plt.xticks([])
#plt.ylim([0, 1])
plt.title("Dirt Muons", fontsize=20)
plt.ylabel("Muon Count/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()


plt.errorbar([1], [N_dirt_out_crt/N_dirt_out], xerr=[0.5], yerr=[0.0], fmt="o", c="blue", label="CRT")
plt.errorbar([1], [N_dirt_out_crt_track/N_dirt_out], xerr=[0.5], yerr=[0.0], fmt="o", c="green", label="CRT (N > 1)")
plt.legend(loc="center", fontsize=14)
plt.xticks([])
plt.ylim([0, 1])
plt.title("Dirt Muons: Endpoint not in TPC", fontsize=20)
plt.ylabel("Muon Fraction/"+str(TOT_POT)+" POT", fontsize=14)
#plt.yscale('log')
plt.show()

In [ ]:

BLAH

In [ ]:
def is_prim_cont(row):
    run, subrun, evt, slc = row["run"], row["subrun"], row["evt"], row["slc"]
    q = "run == "+str(run)+" and subrun == "+str(subrun) + " and evt == "+str(evt) + "and slc == "+str(slc)
    temp_df = particle_df2.query(q)
    x_s, y_s, z_s = temp_df["start_x"].values, temp_df["start_y"].values, temp_df["start_z"].values
    x_e, y_e, z_e = temp_df["end_x"].values, temp_df["end_y"].values, temp_df["end_z"].values
    for num in range(len(x_s)):
        if (x_s[num] > 200 or x_s[num] < -200):
            return 0
        if (x_e[num] > 200 or x_e[num] < -200):
            return 0
        if (y_s[num] > 200 or y_s[num] < -200):
            return 0
        if (y_e[num] > 200 or y_e[num] < -200):
            return 0
        if (z_s[num] > 500 or z_s[num] < 0):
            return 0
        if (z_e[num] > 500 or z_e[num] < 0):
            return 0
        return 1


  
slc_df["primCont"] = slc_df.apply(is_prim_cont, axis=1)    

slc_df[:2]

In [ ]:
# Let's look at the file composition


bin_centers = [0, 1]
bin_width = 1
N_prev_sel = 0
N_prev_rej = 0
for num in range(len(topology_labels.keys())):
    temp_df = slc_df.query(topology_selections[num])
    N_top = temp_df.shape[0]
    N_sel = temp_df.query("primCont == 1").shape[0]
    N_rej = N_top - N_sel
    
    plt.bar(bin_centers, [N_sel, N_rej], width=bin_width, bottom=[N_prev_sel, N_prev_rej], 
            alpha=0.6, label=topology_labels[num])
    N_prev_sel += N_sel
    N_prev_rej += N_rej

#plt.errorbar([1], [N], xerr=[0.5], yerr=np.sqrt(N), fmt="o", c="black", label="All Slices")
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)
plt.legend()
#plt.xticks([])
plt.xticks([0, 1], ['Contained in TPC', 'Not Contained'])
plt.ylabel("Slice Count", fontsize=14)
plt.ylim([0, N])
plt.title("All Primary Particles")
plt.show()